# <center style="color:#145277">Real Time Prediction</center>

In [7]:
# loading dataset
import cv2
import math
import pickle
import warnings
import numpy as np
import mediapipe as mp
warnings.filterwarnings('ignore')

# signs
Sign = ['A', 'B','C','D']

# Function to calculate distance
def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2 + (point1.z - point2.z)**2)

# Load the saved KNN model
with open('sign_gesture.pkl', 'rb') as file:
    knn_model = pickle.load(file)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# Start video capture
cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Calculate distances between landmarks
            distances = []
            for i in range(len(hand_landmarks.landmark)):
                for j in range(i + 1, len(hand_landmarks.landmark)):
                    point1 = hand_landmarks.landmark[i]
                    point2 = hand_landmarks.landmark[j]
                    distance = calculate_distance(point1, point2)
                    distances.append(distance)
            
            # Reshape and scale distances to fit model input
            distances = np.array(distances).reshape(1, -1)
            
            # Predict gesture using the loaded model
            gesture = knn_model.predict(distances)
            
            # Display the gesture on the image
            cv2.putText(img, f'Gesture: {Sign[gesture[0]]}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (2, 102, 47), 2, cv2.LINE_AA)

    cv2.imshow("Sign Gesture", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()